In [ ]:
import pandas as pd
import numpy as np
import math
from pandas import DataFrame
dataset = pd.read_csv('redfin_data.csv')
print(dataset.columns)

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')


In [ ]:
# have a list of all the house's latitudes
latlist = dataset["LATITUDE"]
# have a list of all the house's longitudes
longlist = dataset["LONGITUDE"]

# read the hospital longitudes/latitudes
hospital_df = pd.read_csv('Baltimore_Hospital1_test.csv')

grocery_df = pd.read_csv('Baltimore_Grocery1_test.csv')

parks_df = pd.read_csv('Baltimore_Parks1_test.csv')

whole_traders_df = pd.read_csv('Baltimore_Whole_Traders_test.csv')


In [ ]:
hospital_df

,Address,Latitude,Longitude
0,2401 W Belvedere Ave Baltimore MD 21215,39.352329,-76.663397
1,201 E University Pkwy Baltimore MD 21218,39.329597,-76.614320
2,600 N Wolfe St Baltimore MD 21287,39.297252,-76.592210
3,3001 S Hanover St Baltimore MD 21225,39.252107,-76.614284
4,900 S Caton Ave Baltimore MD 21229,39.271835,-76.674208
5,5101 Lanier Ave Baltimore MD 21215,39.352307,-76.664067
6,22 S Greene St Baltimore MD 21201,39.288013,-76.624845
7,4940 Eastern Ave Baltimore MD 21224,39.290162,-76.547107
8,227 St Paul Pl Baltimore MD 21202,39.292202,-76.613297
9,5601 Loch Raven Blvd Baltimore MD 21239,39.357996,-76.587110


In [ ]:
#SAMPLE INPUT
coordsList = []
for i in range(len(dataset)):
  coordsList.append([(dataset['LATITUDE'][i]), (dataset['LONGITUDE'][i])])
originsCoords = coordsList

In [ ]:
class NonTraditionalRating():
  def __init__(self, originsCoords, hospital_df):
    self.originsCoords = originsCoords
    self.info_df = hospital_df
    self.final_pos = self.info_df["Address"]
    self.final_lat = self.info_df["Latitude"]
    self.final_long = self.info_df["Longitude"]
    self.matrix_df = self.getmatrix()
    #self.matrix_df.index = self.startpos
    #self.matrix_df.columns = self.final_pos
    self.topFive = self.topFive_df()


  def getmatrix(self):
    distance_df = pd.DataFrame()
    for i in range(len(self.originsCoords)):
      distList = []
      for j in range(len(self.final_long)):
        distList.append(self.LatLongDist(self.originsCoords[i][0], self.originsCoords[i][1], self.final_lat[j], self.final_long[j]))
      distance_df = distance_df.append([distList])
    return distance_df

      
  def topFive_df(self):
    topFive_df = pd.DataFrame()
    self.matrix_df.values
    for i in range(len(self.matrix_df.values)):
      topFive = list(np.sort(self.matrix_df.values[i])[0:1])
      topFive_df = topFive_df.append([topFive])
    #topFive_df.index = self.startpos
    return topFive_df
  
  def LatLongDist(self, lat1, long1, lat2, long2):
    A = math.radians(float(lat1))
    B = math.radians(float(long1))
    C = math.radians(lat2)
    D = math.radians(long2)
    dlon = D - B
    dlat = C - A
    a = math.sin(dlat / 2)**2 + math.cos(A) * math.cos(C) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return 6373.0 * c

In [ ]:
import math
hospital = NonTraditionalRating(originsCoords, hospital_df)
best_dist = hospital.topFive_df()
best_hospital = best_dist.rename(columns={0: 'ClosestHospital'})
dataset["ClosestHospital"] = best_hospital['ClosestHospital'].values

In [ ]:
grocery = NonTraditionalRating(originsCoords, grocery_df)
best_dist = grocery.topFive_df()
best_grocery = best_dist.rename(columns={0: 'ClosestGrocery'})
dataset["ClosestGrocery"] = best_grocery['ClosestGrocery'].values

In [ ]:
parks = NonTraditionalRating(originsCoords, parks_df)
best_dist = parks.topFive_df()
best_parks = best_dist.rename(columns={0: 'ClosestParks'})
dataset["ClosestParks"] = best_parks['ClosestParks'].values

In [ ]:
whole = NonTraditionalRating(originsCoords, whole_traders_df)
best_dist = whole.topFive_df()
best_whole = best_dist.rename(columns={0: 'ClosestWhole'})
dataset["ClosestWhole"] = best_whole['ClosestWhole'].values

In [ ]:
dataset.to_csv('updated.csv', index=True,)
